# Kelvin wave in a 2D periodic channel

by Marvin Lorenz and Markus Reinert, February to May 2021

The following “magic” command enables interactive plots.
This requires https://github.com/matplotlib/ipympl.
Alternatively you could use `%matplotlib notebook` (deprecated).

In [1]:
%matplotlib widget

In [2]:
import datetime

import numpy as np
import matplotlib.pyplot as plt

import pygetm
import pygetm.domain

In [3]:
L = 500e3
H = 10.0
f = 0.0001

# Set up a 2D-domain that is periodic in x-direction
domain = pygetm.domain.Domain.create_cartesian(
    L * np.arange(100) / 100,
    L * np.arange(100) / 100,
    1,
    periodic_x=True,
    H=H,
    f=f,
)
sim = pygetm.Simulation(domain, runtype=1, advection_scheme=1)

# No surface forcing
tausx, tausx_ = domain.T.array(fill=0.0)
tausy, tausy_ = domain.T.array(fill=0.0)
sp, sp_ = domain.T.array(fill=0.0)

# Define, extract, and/or calculate physical constants
g = 9.81
c = np.sqrt(g * H)
print("Phase speed of long gravity waves: {:.2f} m/s".format(c))
print("Coriolis parameter: f = {:.5f} / s".format(f))
a = c / f
print("External Rossby radius: {:.2f} km".format(a / 1e3))

# Set initial conditions for an almost linear Kelvin wave
eta_0 = 0.001
k = 2 * 2 * np.pi / L
l = 2 * np.pi / k
print("Wave length: {:.2f} km".format(l / 1e3))
omega = c * k
print("Period: {:.0f} minutes = {:.1f} hours".format(2*np.pi/omega / 60, 2*np.pi/omega / 3600))
domain.T.z[...] = (
    eta_0
    * np.exp(-domain.T.yi[1:, 1:] / a)
    * np.sin(k * domain.T.xi[1:, 1:])
    * domain.T.mask
)
sim.momentum.U[...] = (
   eta_0 * c
   * np.exp(-domain.T.yi[1:, 1:] / a)
   * np.sin(k * domain.T.x)
   * domain.U.mask
)
# Keep only one wavelength
sim.momentum.U[domain.T.x > l] = 0
domain.T.z[domain.T.xi[1:, 1:] > l] = 0

# Set the time-stepping
start = datetime.datetime(2020, 1, 1, 0, 0)
stop = datetime.datetime(2020, 1, 2, 4, 0)
timestep = 60.0
time = start

# Set up a figure showing surface elevation and velocity vectors
nsample = 4
fig, ax = plt.subplots()
title = ax.set_title(time.strftime('Day: %j, Time: %H:%M:%S'))
ax.set_xlabel("x (m)")
ax.set_ylabel("y (m)")
pc = ax.pcolormesh(domain.T.xi, domain.T.yi, domain.T.z, vmin=-eta_0, vmax=eta_0, cmap="seismic")
cb = fig.colorbar(pc, label='elevation (m)')
Q = ax.quiver(
    domain.T.x[::nsample, ::nsample],
    domain.T.y[::nsample, ::nsample],
    sim.momentum.U[::nsample, ::nsample],
    sim.momentum.V[::nsample, ::nsample],
    scale=0.1,
)
l = ax.axvline(0, color="black")
ax.set_xlim(None, L)
print("The black line should always be at the same phase of the Kelvin wave")

Phase speed of long gravity waves: 9.90 m/s
Coriolis parameter: f = 0.00010 / s
External Rossby radius: 99.05 km
Wave length: 250.00 km
Period: 421 minutes = 7.0 hours


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

The black line should always be at the same phase of the Kelvin wave


In [ ]:
dist_U = domain.distribute(sim.momentum.U_)
dist_V = domain.distribute(sim.momentum.V_)
dist_U.update_halos()
dist_V.update_halos()

plotting_interval = 10
istep = 0
while time <= stop:
    sim.pressure.surface(domain.T.z_, sp_)
    sim.momentum.uv_momentum_2d(timestep, tausx_, tausy_, sim.pressure.dpdx_, sim.pressure.dpdy_)
    dist_U.update_halos()
    dist_V.update_halos()
    sim.sealevel.update(timestep, sim.momentum.U_, sim.momentum.V_)
    sim.update_depth()

    if istep % plotting_interval == 0:
        print(istep, time)
        Q.set_UVC(sim.momentum.U[::nsample, ::nsample], sim.momentum.V[::nsample, ::nsample])
        title.set_text(time.strftime('Day: %j, Time: %H:%M:%S'))
        pc.set_array(domain.T.z.ravel())
        l.remove()
        l = ax.axvline((omega * (time - start).total_seconds() / k) % L, color="black")
        fig.canvas.draw()

    istep += 1
    time += datetime.timedelta(seconds=timestep)

0 2020-01-01 00:00:00
10 2020-01-01 00:10:00
20 2020-01-01 00:20:00
30 2020-01-01 00:30:00
40 2020-01-01 00:40:00
50 2020-01-01 00:50:00
60 2020-01-01 01:00:00
70 2020-01-01 01:10:00
80 2020-01-01 01:20:00
90 2020-01-01 01:30:00
100 2020-01-01 01:40:00
110 2020-01-01 01:50:00
120 2020-01-01 02:00:00
130 2020-01-01 02:10:00
140 2020-01-01 02:20:00
150 2020-01-01 02:30:00
160 2020-01-01 02:40:00
170 2020-01-01 02:50:00
180 2020-01-01 03:00:00
190 2020-01-01 03:10:00
200 2020-01-01 03:20:00
210 2020-01-01 03:30:00
220 2020-01-01 03:40:00
230 2020-01-01 03:50:00
240 2020-01-01 04:00:00
250 2020-01-01 04:10:00
260 2020-01-01 04:20:00
270 2020-01-01 04:30:00
280 2020-01-01 04:40:00
290 2020-01-01 04:50:00
300 2020-01-01 05:00:00
310 2020-01-01 05:10:00
320 2020-01-01 05:20:00
330 2020-01-01 05:30:00
340 2020-01-01 05:40:00
350 2020-01-01 05:50:00
360 2020-01-01 06:00:00
370 2020-01-01 06:10:00
380 2020-01-01 06:20:00
390 2020-01-01 06:30:00
400 2020-01-01 06:40:00
410 2020-01-01 06:50:00
420